In [2]:
import numpy as np
import pandas as pd
import pyodbc
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib
import csv

E:\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
def lift(df,Bin):
    #df is sorted
    n=int(df.shape[0]/Bin)
    df['bin']=0
    for i in range(Bin):
        a=int(i*n)
        b=int((i+1)*n)
        if i<Bin-1:
            df['bin'][a:b]=i
        else:
            df['bin'][a:]=i
    #return df[['Prequal4mos','bin']].groupby('bin').agg(['count','sum','mean'])

# Step 1 - PIF Model

In [4]:
DataPath=os.path.normpath("S:\Risk Management\Modeling\Projects\Marketing\PurchPropensity\B - Data Exploration\DevDataText_pifsample20190311.csv")
df=pd.read_csv(DataPath, low_memory=False)
df['DV_PIF']=0
df['DV_PIF'][df['PIF_2mos']==1]=1
df['DV_PIF'][df['PIF_3to6mos']==1]=2
df_train=df[df['sample']=='DEV']
df_test=df[df['sample']=='VAL']

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [5]:
# df_train=shuffle(df_train,random_state=123)

vectorizer = TfidfVectorizer(ngram_range = ( 1, 4 ), stop_words='english')
vectorizer.fit(df_train['totaltranscript'])

train_text_features=vectorizer.transform(df_train['totaltranscript'])
test_text_features=vectorizer.transform(df_test['totaltranscript'])

In [6]:
# Could directly use index instead of feature_names
feature_names=vectorizer.get_feature_names()
Pval_2mos=list(zip(feature_names, chi2(train_text_features, df_train['PIF_2mos'])[1]))
Pval_3to6mos=list(zip(feature_names, chi2(train_text_features, df_train['PIF_3to6mos'])[1]))

Pval_2mos.sort(key=lambda t:t[1])
Pval_3to6mos.sort(key=lambda t:t[1])

In [17]:
Pval_2mos[:700]

[('payoff', 4.605419418208697e-169),
 ('payoff quote', 1.2346302670316665e-53),
 ('quote', 6.004932589220548e-38),
 ('payoff statement', 6.430246879072657e-32),
 ('fax', 1.0246447876744895e-20),
 ('request payoff', 1.2607215584766032e-19),
 ('payoff request', 6.341873471918438e-16),
 ('closing', 8.538285879461101e-16),
 ('authorization', 9.526839926685062e-16),
 ('fax number', 1.1555012192524195e-13),
 ('request', 1.2616594662990588e-13),
 ('wire', 5.328602213757624e-12),
 ('title', 6.5970350904386366e-12),
 ('payoff okay', 7.923935077014876e-12),
 ('cashier check', 3.1887462539349755e-11),
 ('cashier', 4.1208488450161007e-11),
 ('send payoff', 9.822048145908259e-11),
 ('instructions', 2.624223983241538e-10),
 ('payoff code', 7.1075714047205e-10),
 ('title company', 1.1373912819579283e-09),
 ('requested', 1.3754678582575209e-09),
 ('hours', 2.2403280649539374e-09),
 ('requested payoff', 4.077258504303421e-09),
 ('receive payoff', 7.459724430614155e-09),
 ('order payoff', 1.852751259130

In [18]:
train_selected_features=list(set([t[0] for t in Pval_2mos[:700]]+[t[0] for t in Pval_3to6mos[:500]]))

In [19]:
train_selected_features_index=[feature_names.index(x) for x in train_selected_features]

In [110]:
train_text_features[:,[1,3]]

<1151755x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [10]:
len(train_selected_features_index)

6859

In [33]:
drop=[]

In [120]:
#for loss in ['deviance', 'exponential']:
loss='deviance'
depth=10
n=300
lr=0.01
#err=[]
# for depth in [3,6,9,12]:
# for depth in range(3,12):
#     for n in [200,300,400,500]:
#for depth in [4,5]:
#for lr in [0.02]:
gbclf=GradientBoostingClassifier(loss=loss, n_estimators=n, max_depth=depth, subsample=0.5, random_state=123,verbose=1,
                                 learning_rate=lr)
gbclf.fit(train_text_features[:,train_selected_features_index],df_train['DV_PIF'])
print(depth)
print(n)
print(gbclf.score(train_text_features[:,train_selected_features_index],df_train['DV_PIF']))
print(gbclf.score(test_text_features[:,train_selected_features_index],df_test['DV_PIF']))

      Iter       Train Loss      OOB Improve   Remaining Time 
         1       22651.9868         192.3517            5.54m
         2       22453.1370         189.2276            5.74m
         3       22236.5794         185.3637            5.87m
         4       22110.4520         182.2659            5.90m
         5       21885.9408         178.2099            5.95m
         6       21684.6066         174.3651            5.99m
         7       21533.3067         170.8274            5.98m
         8       21261.5764         167.3688            5.98m
         9       21199.7666         164.5750            5.95m
        10       21026.4365         160.1395            5.92m
        20       19409.2194         132.4265            5.70m
        30       18191.5966         109.5309            5.46m
        40       17088.4705          91.0971            5.23m
        50       16184.0543          75.2973            5.04m
        60       15597.5317          62.5089            4.81m
       

In [ ]:
0.8922193770815551
0.8752067417500197

In [111]:
importances=zip(gbclf.feature_importances_,train_selected_features_index)
ListImp=list(importances)
# for x in sorted(ListImp):
#     print(feature_names[x[1]])

In [110]:
train_selected_features_index

[19156864,
 6627552,
 18949929,
 1656180,
 26878532,
 3905727,
 14412242,
 5324488,
 24116903,
 3174288,
 15099698,
 17036970,
 18485477,
 4577953,
 1031615,
 2573877,
 1714187,
 23733268,
 22135326,
 6810935,
 1199143,
 9569685,
 15876238,
 17040263,
 25146848,
 13145954,
 18383791,
 8595118,
 17044267,
 18379259,
 18485149,
 16489299,
 14453600,
 23280490,
 20510607,
 24971071,
 4783701,
 17042678,
 8575121,
 6804252,
 6814073,
 19179259,
 17019817,
 8724854,
 7031400,
 8616401,
 8758745,
 18387620,
 15837805,
 17020675,
 15598039,
 18294978,
 1509138,
 5165414,
 9167529,
 835725,
 22875440,
 17944678,
 960038,
 10923958,
 10923846,
 4761687,
 11405896,
 15674468,
 23940181,
 8434366,
 6807104,
 25676102,
 3622910,
 25628386,
 19455361,
 18757346,
 23675797,
 995116,
 17678751,
 10921299,
 6802009,
 13144765,
 26844895,
 17039397,
 25628193,
 24966519,
 17701953,
 13146950,
 24331735,
 14411947,
 18382286,
 11338764,
 17038429,
 18383543,
 19138067,
 14541929,
 3649013,
 2390985,
 24

In [112]:
drop=list(set(drop+[x[1] for x in ListImp if x[0]<1e-4]))
train_selected_features_index=[x for x in train_selected_features_index if x not in drop]

In [113]:
len(drop)

668

In [119]:
len(train_selected_features_index)

532

In [155]:
train_selected_features_index

[17028682,
 26069251,
 16222136,
 7239073,
 14866707,
 21610365,
 15837806,
 19138067,
 18382286,
 5261348,
 25062612,
 23836669,
 25628386,
 3645973,
 18757086,
 22646515,
 19924856,
 4786512,
 9019675,
 8777180,
 16070292,
 20510607,
 25741115,
 24331735,
 18482648,
 14069466,
 19455360,
 4050473,
 18520425,
 17038809,
 24658995,
 4998417,
 26844895,
 19572095,
 18472877,
 17020486,
 11390773,
 4871881,
 16080728,
 13284586,
 17022133,
 26105630,
 20403956,
 16212951,
 7450033,
 17030288,
 24659475,
 17037238,
 18914401,
 17026021,
 2310658,
 22651245,
 1411942,
 20420191,
 17701953,
 25740647,
 24367511,
 17028158,
 18782708,
 2946175,
 4225064,
 17033371,
 1648549,
 3905727,
 6508285,
 25628193,
 8718716,
 6290511,
 2075111,
 14400038,
 5164081,
 24451864,
 18711034,
 9653677,
 7867674,
 1700087,
 10250121,
 1535355,
 22103609,
 20420192,
 26069168,
 17028455,
 17022958,
 25670026,
 11816693,
 18099132,
 14453600,
 17044267,
 19491840,
 2687257,
 19155363,
 13143876,
 4998416,
 177

In [114]:
cwd=os.getcwd()
joblib.dump(vectorizer, os.path.join(cwd,'out/PIF_Vectorizer.pkl'))
joblib.dump(gbclf, os.path.join(cwd,'out/PIF_GB_text.pkl'))

['Y:\\VR2FS1\\Risk Management\\Modeling\\Projects\\Marketing\\PurchPropensity\\C - Model Selection\\v3\\PIF_GB_text.pkl']

In [ ]:
input_features=list(zip([feature_names[i] for i in train_selected_features_index], train_selected_features_index))

In [ ]:
with open(os.path.join(cwd,'out/PIF_input_text.csv'), 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(input_features)

In [95]:
import gc
gc.collect()

1204

In [121]:
pred=gbclf.predict_proba(test_text_features[:,train_selected_features_index])
df_test['prob_2mos']=pred[:,1]
df_test['prob_3to6mos']=pred[:,2]
score=df_test[['prob_2mos','prob_3to6mos','PIF_3to6mos','PIF_2mos']].copy()
score.sort_values('prob_3to6mos',ascending=False, inplace=True)
print(score[:int(2500)].mean())

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


prob_2mos       0.310018
prob_3to6mos    0.144618
PIF_3to6mos     0.113200
PIF_2mos        0.296000
dtype: float64


In [31]:
score.sort_values('prob_3to6mos',ascending=False, inplace=True)
lift(score,10)
score[['PIF_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


PIF_3to6mos               
          count  sum      mean
bin                           
0          2539  233  0.091768
1          2539  178  0.070106
2          2539  174  0.068531
3          2539  151  0.059472
4          2539  144  0.056715
5          2539  124  0.048838
6          2539  117  0.046081
7          2539  108  0.042536
8          2539  105  0.041355
9          2543   79  0.031066

In [123]:
score.sort_values('prob_3to6mos',ascending=False, inplace=True)
lift(score,10)
score[['PIF_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


PIF_3to6mos               
          count  sum      mean
bin                           
0          2539  286  0.112643
1          2539  222  0.087436
2          2539  164  0.064592
3          2539  170  0.066955
4          2539  140  0.055140
5          2539  139  0.054746
6          2539  115  0.045293
7          2539  103  0.040567
8          2539  109  0.042930
9          2543   87  0.034212

In [122]:
score.sort_values('prob_2mos',ascending=False, inplace=True)
lift(score,10)
score[['PIF_2mos','bin']].groupby('bin').agg(['count','sum','mean'])

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


PIF_2mos                
       count   sum      mean
bin                         
0       2539  1270  0.500197
1       2539   235  0.092556
2       2539   149  0.058685
3       2539   129  0.050807
4       2539   105  0.041355
5       2539    82  0.032296
6       2539    87  0.034265
7       2539    80  0.031508
8       2539    74  0.029145
9       2543    54  0.021235

# Step 2 - Purchase Refi Model

In [128]:
DataPath=os.path.normpath("S:\Risk Management\Modeling\Projects\Marketing\PurchPropensity\B - Data Exploration\DevDataTextPR_sample20190311.csv")
df_PR=pd.read_csv(DataPath, low_memory=False)
df_PR_train=df_PR[(df_PR['PIF_3to6mos']==1) & ((df_PR['PURPOSE']=='Purchase') | (df_PR['PURPOSE']=='Refi')) & (df_PR['sample']=='DEV')]
df_PR_test=df_PR[(df_PR['PIF_3to6mos']==1) & ((df_PR['PURPOSE']=='Purchase') | (df_PR['PURPOSE']=='Refi')) & (df_PR['sample']=='VAL')]

In [129]:
df_PR_train.groupby('PURPOSE').count()

,LoanNumber,totaltranscript,PIF_2mos,PIF_3to6mos,PIF_P_3to6mos,sample
PURPOSE,,,,,,
Purchase,17219,17219,17219,17219,17219,17219
Refi,30548,30548,30548,30548,30548,30548


In [131]:
df_PR_test['PIF_P_3to6mos'].describe()

count    20436.000000
mean         0.358485
std          0.479567
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: PIF_P_3to6mos, dtype: float64

In [130]:
vectorizer = TfidfVectorizer(ngram_range = ( 1, 4 ), stop_words='english')
vectorizer.fit(df_PR_train['totaltranscript'])

PR_train_text=vectorizer.transform(df_PR_train['totaltranscript'])
PR_test_text=vectorizer.transform(df_PR_test['totaltranscript'])

feature_names=vectorizer.get_feature_names()

In [132]:
Pval_Purchase=list(zip(range(len(feature_names)), chi2(PR_train_text, df_PR_train['PIF_P_3to6mos'])[1]))
Pval_Purchase.sort(key=lambda t:t[1])

In [186]:
PR_train_text_index=[x[0] for x in Pval_Purchase[:500]]

In [185]:
drop=[]

In [228]:
#for loss in ['deviance', 'exponential']:
loss='exponential'
depth=7
n=400
lr=0.02
#err=[]
# for n in [200, 300, 400, 500]:
#     for lr in [0.1, 0.05, 0.03, 0.02, 0.01]:
#         for depth in range(3,12):
gbclf=GradientBoostingClassifier(loss=loss, n_estimators=n, max_depth=depth, subsample=0.5, random_state=123,
                                 verbose=0, learning_rate=lr)
gbclf.fit(PR_train_text[:,PR_train_text_index],df_PR_train['PIF_P_3to6mos'])
print('n_estimators='+str(n))
print('Max_depth='+str(depth))
print('Learning Rate='+str(lr))
print(gbclf.score(PR_train_text[:,PR_train_text_index],df_PR_train['PIF_P_3to6mos']))
print(gbclf.score(PR_test_text[:,PR_train_text_index],df_PR_test['PIF_P_3to6mos']))
print("")

n_estimators=400
Max_depth=7
Learning Rate=0.02
0.7265894864655515
0.6917694265022509



In [ ]:
0.7265894864655515
0.6917694265022509

In [225]:
importances=zip(gbclf.feature_importances_,PR_train_text_index)
ListImp=list(importances)
sorted(ListImp)
#for x in sorted(ListImp):
#    print(feature_names[x[1]])

[(3.361322627807829e-05, 8705109),
 (3.48846426925703e-05, 13423915),
 (3.680274560261673e-05, 6394334),
 (5.107597914522884e-05, 1630964),
 (5.138640217256937e-05, 6638322),
 (5.157934815194885e-05, 16570953),
 (5.878272342720485e-05, 16561479),
 (8.915262910510776e-05, 6394301),
 (9.682316291685258e-05, 12445848),
 (0.00010299993014909404, 8729276),
 (0.00011528082396758056, 6701562),
 (0.00011768365503247591, 17085861),
 (0.00011905483174329968, 621506),
 (0.000128330814779117, 5934945),
 (0.00014733362226353875, 1630943),
 (0.0001518568569035875, 23563797),
 (0.00015408997422667497, 17389670),
 (0.00016246503104332496, 4857671),
 (0.00016253573753775868, 23716230),
 (0.00016492262168632437, 6394249),
 (0.00016730457699208268, 7681163),
 (0.00017288403948600376, 6767709),
 (0.00017648130422439067, 16108773),
 (0.00017887608722233955, 13423904),
 (0.00017889526463282522, 20582923),
 (0.00018127769216214632, 16629720),
 (0.00018547469061663658, 17712331),
 (0.0001870976893878369, 6638

In [226]:
PR_train_text_index=PR_train_text_index_old

In [222]:
drop=list(set(drop+[x[1] for x in ListImp if x[0]<1e-4]))
PR_train_text_index=[x for x in PR_train_text_index if x not in drop]

In [227]:
len(PR_train_text_index)

449

In [257]:
pred=gbclf.predict_proba(PR_test_text[:,PR_train_text_index])
df_PR_test['prob_P_3to6mos']=pred[:,1]
score=df_PR_test[['PIF_P_3to6mos','prob_P_3to6mos']].copy()
score.sort_values('prob_P_3to6mos',ascending=False, inplace=True)
print(score[:int(1800)].mean())

PIF_P_3to6mos     0.722778
prob_P_3to6mos    0.707543
dtype: float64


E:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [197]:
print(score[:int(1800)].mean())

PIF_P_3to6mos     0.383183
prob_P_3to6mos    0.370468
dtype: float64


In [228]:
score=df_PR_test[['PIF_P_3to6mos','prob_P_3to6mos']].copy()
score.sort_values('prob_P_3to6mos',ascending=False, inplace=True)
lift(score,10)
score[['PIF_P_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


PIF_P_3to6mos                
            count   sum      mean
bin                              
0            1772  1272  0.717833
1            1772   919  0.518623
2            1772   820  0.462754
3            1772   744  0.419865
4            1772   690  0.389391
5            1772   637  0.359481
6            1772   620  0.349887
7            1772   538  0.303612
8            1772   341  0.192438
9            1772   209  0.117946

In [230]:
cwd=os.getcwd()
joblib.dump(vectorizer, 'out/Purchase_Vectorizer.pkl')
joblib.dump(gbclf, os.path.join(cwd,'out/Purchase_GB_text.pkl'))

['Y:\\VR2FS1\\Risk Management\\Modeling\\Projects\\Marketing\\PurchPropensity\\C - Model Selection\\v3\\out/Purchase_GB_text.pkl']

In [231]:
input_features=list(zip([feature_names[i] for i in PR_train_text_index], PR_train_text_index))
with open('out/Purchase_input_text.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(input_features)

## Remove Sensitive Words/Phrases

In [257]:
sens=['african','american','american indian',#'american title',
      'asian','asian black',
      'birth',
      'black','black african',
      'citizen',
      'consider american','date birth',
      'death','death certificate',
      'discriminate','discriminate based',
      'female',
      'gender',
      'hispanic','indian','indian alaskan','male','male female','race',
      'married','mortgage state',
      'passed away','residence','single','usa',
      'white','wife'
     ]

In [265]:
len(sens)

31

In [263]:
with open('out/PIF_input_text old.csv',newline='') as csvfile:
    reader=csv.reader(csvfile)
    List_index=list()
    for row in reader:
        List_index=List_index+row
List_index=[eval(x) for x in List_index]
train_selected_features_index=[x[1] for x in List_index if x[0] not in sens]

with open('out/Purchase_input_text old.csv',newline='') as csvfile:
    reader=csv.reader(csvfile)
    List_index=list()
    for row in reader:
        List_index=List_index+row
List_index=[eval(x) for x in List_index]
PR_train_text_index=[x[1] for x in List_index if x[0] not in sens]

In [264]:
len(PR_train_text_index)

418

In [275]:
drop=[]

In [284]:
#################################################
#Sensitive words appeared in purchase model only# 
#################################################
loss='exponential'
depth=7
n=400
lr=0.02
gbclf=GradientBoostingClassifier(loss=loss, n_estimators=n, max_depth=depth, subsample=0.5, random_state=123,
                                 verbose=0, learning_rate=lr)
gbclf.fit(PR_train_text[:,PR_train_text_index],df_PR_train['PIF_P_3to6mos'])
print(gbclf.score(PR_train_text[:,PR_train_text_index],df_PR_train['PIF_P_3to6mos']))
print(gbclf.score(PR_test_text[:,PR_train_text_index],df_PR_test['PIF_P_3to6mos']))

0.7277409089957502
0.6896652965355256


In [ ]:
0.7273640798040488
0.6884908984145626

In [282]:
importances=zip(gbclf.feature_importances_,PR_train_text_index)
ListImp=list(importances)
drop=list(set(drop+[x[1] for x in ListImp if x[0]<1e-6]))
PR_train_text_index=[x for x in PR_train_text_index if x not in drop]

In [283]:
len(PR_train_text_index)

414

In [285]:
joblib.dump(gbclf, os.path.join(cwd,'out/Purchase_GB_text.pkl'))

['Y:\\VR2FS1\\Risk Management\\Modeling\\Projects\\Marketing\\PurchPropensity\\C - Model Selection\\v3\\out/Purchase_GB_text.pkl']

In [286]:
input_features=list(zip([feature_names[i] for i in PR_train_text_index], PR_train_text_index))
with open('out/Purchase_input_text.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(input_features)

# Score Calls

In [232]:
DataPath=os.path.normpath("S:\Risk Management\Modeling\Projects\Marketing\PurchPropensity\B - Data Exploration\TextSample20190314.csv")
text_df=pd.read_csv(DataPath, low_memory=False)

In [238]:
text_df.shape

(96817, 14)

In [239]:
text_df.EomDate.unique()

array(['31DEC2017', '31JAN2018', '28FEB2018', '31MAR2018'], dtype=object)

In [252]:
gbclf_PIF=joblib.load('out/PIF_GB_text.pkl')
vectorizer=joblib.load('out/PIF_Vectorizer.pkl')

In [291]:
with open('out/PIF_input_text.csv',newline='') as csvfile:
    reader=csv.reader(csvfile)
    List_index=list()
    for row in reader:
        List_index=List_index+row
List_index=[eval(x) for x in List_index]
train_selected_features_index=[x[1] for x in List_index]

In [287]:
gbclf_PR=joblib.load('out/Purchase_GB_text.pkl')
vectorizer_PR=joblib.load('out/Purchase_Vectorizer.pkl')

In [289]:
with open('out/Purchase_input_text.csv',newline='') as csvfile:
    reader=csv.reader(csvfile)
    List_index=list()
    for row in reader:
        List_index=List_index+row
List_index=[eval(x) for x in List_index]
PR_train_text_index=[x[1] for x in List_index]

In [240]:
PIF_text_features=vectorizer.transform(text_df['totaltranscript'])
Purchase_text_features=vectorizer_PR.transform(text_df['totaltranscript'])

In [292]:
PIF_pred=gbclf_PIF.predict_proba(PIF_text_features[:,train_selected_features_index])
Purchase_pred=gbclf_PR.predict_proba(Purchase_text_features[:,PR_train_text_index])

text_df['TextPIF2mosProb']=PIF_pred[:,1]
text_df['TextPIF3to6mosProb']=PIF_pred[:,2]
text_df['TextPurchase3to6mosProb']=Purchase_pred[:,1]
text_df['TextPIFPurchase3to6mosProb']=text_df['TextPIF3to6mosProb']*text_df['TextPurchase3to6mosProb']

In [11]:
text_df.columns

Index(['LoanNumber', 'InteractionKey', 'CallDate', 'EomDate',
       'totaltranscript', 'EomPrior', 'PURPOSE', 'PayoffDate', 'Purchase',
       'Refi', 'PIF_2mos', 'PIF_3to6mos', 'PIF_P_3to6mos', 'sample',
       'TextPIF2mosProb', 'TextPIF3to6mosProb', 'TextPurchase3to6mosProb',
       'TextPIFPurchase3to6mosProb'],
      dtype='object')

In [294]:
text_df[['LoanNumber', 'CallDate','TextPIF2mosProb', 'TextPIF3to6mosProb', 'TextPurchase3to6mosProb',
         'TextPIFPurchase3to6mosProb']].to_csv('TextScores.csv',index=False)

In [16]:
score=text_df.sort_values('TextPIF3to6mosProb',ascending=False)[['PIF_P_3to6mos','PIF_3to6mos','TextPIF3to6mosProb','PURPOSE']]
lift(score,10)
score[['PIF_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


PIF_3to6mos                
          count   sum      mean
bin                            
0         98357  3962  0.040282
1         98357  3734  0.037964
2         98357  2908  0.029566
3         98357  2829  0.028763
4         98357  2310  0.023486
5         98357  1867  0.018982
6         98357  1314  0.013359
7         98357   635  0.006456
8         98357   988  0.010045
9         98358  1206  0.012261

In [20]:
score=text_df.sort_values('TextPIFPurchase3to6mosProb',ascending=False)[['PIF_P_3to6mos','PIF_3to6mos','TextPIFPurchase3to6mosProb','PURPOSE']]
#score=score[((score['PURPOSE']=='Purchase') | (score['PURPOSE']=='Refi'))]
lift(score,10)
score[['PIF_P_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


PIF_P_3to6mos                
            count   sum      mean
bin                              
0           98357  4147  0.042163
1           98357  2003  0.020365
2           98357  1522  0.015474
3           98357  1064  0.010818
4           98357   749  0.007615
5           98357  1000  0.010167
6           98357   947  0.009628
7           98357   890  0.009049
8           98357   849  0.008632
9           98358   627  0.006375

In [255]:
score=text_df.sort_values('TextPIFPurchase3to6mosProb',ascending=False)[['PIF_P_3to6mos','PIF_3to6mos','TextPIFPurchase3to6mosProb','PURPOSE']]
#score=score[((score['PURPOSE']=='Purchase') | (score['PURPOSE']=='Refi'))]
lift(score,10)
score[['PIF_P_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


PIF_P_3to6mos               
            count  sum      mean
bin                             
0            9681  121  0.012499
1            9681   62  0.006404
2            9681   46  0.004752
3            9681   57  0.005888
4            9681   34  0.003512
5            9681   38  0.003925
6            9681   35  0.003615
7            9681   32  0.003305
8            9681   21  0.002169
9            9688   32  0.003303

In [293]:
score=text_df.sort_values('TextPIFPurchase3to6mosProb',ascending=False)[['PIF_P_3to6mos','PIF_3to6mos','TextPIFPurchase3to6mosProb','PURPOSE']]
#score=score[((score['PURPOSE']=='Purchase') | (score['PURPOSE']=='Refi'))]
lift(score,10)
score[['PIF_P_3to6mos','bin']].groupby('bin').agg(['count','sum','mean'])

E:\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
E:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


PIF_P_3to6mos               
            count  sum      mean
bin                             
0            9681  125  0.012912
1            9681   58  0.005991
2            9681   54  0.005578
3            9681   46  0.004752
4            9681   40  0.004132
5            9681   43  0.004442
6            9681   25  0.002582
7            9681   32  0.003305
8            9681   24  0.002479
9            9688   31  0.003200

In [302]:
PIF_pred[:5,2]*PIF_pred[:5,1]

array([0.00441992, 0.00456732, 0.00444812, 0.00570339, 0.00258838])

In [301]:
gbclf_PIF.classes_

array([0, 1, 2], dtype=int64)